In [ ]:
!pip install tensorflow
!pip install tensorflow-probability
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
dados = pd.read_excel('/content/Consumo.xlsx')
#Exibe as cinco primeiras linhas
dados.head()

In [ ]:
#Normalizando os dados - não é obrigatório mas é uma boa prática
from sklearn.preprocessing import StandardScaler
normalizador = StandardScaler()

dados[['cylinders', 'displacement', 'horsepower','weight', 'acceleration', 'year', \
     'origin']] = normalizador.fit_transform(dados[['cylinders','displacement','horsepower',\
                                                  'weight','acceleration','year', 'origin']])

In [ ]:
# Definir as variáveis independentes (X) e a variável dependente (y)
X = dados[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']]
y = dados['mpg']

In [ ]:
# Separa a amostra em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Garantir que X seja um array NumPy
X_test = np.array(X_test)
X_train = np.array(X_train)




In [ ]:
# Modelo de Regressão Bayesiana com TFP
def build_bayesian_linear_regression_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1 + input_shape, activation=None, kernel_initializer='zeros', bias_initializer='zeros'),
        tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :1], scale=1e-3 + tf.math.softplus(0.05 * t[..., 1:]))),
    ])
    return model
    ## Função para construir o modelo de regressão linear bayesiana
def build_bayesian_linear_regression_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, input_shape=(input_shape,), activation=None,
                              kernel_initializer='zeros', bias_initializer='zeros'),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1)),
    ])
    return model

# Construção e compilação do modelo
model = build_bayesian_linear_regression_model(X_train.shape[1])
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss=lambda y, rv_y: -rv_y.log_prob(y))

# Treinamento do modelo
history = model.fit(X_train, y_train, epochs=1000, verbose=0)

# Previsões
y_pred = model.predict(X_test).squeeze()  # Remover dimensão extra, se houver

# Previsões
y_pred_t = model.predict(X_train).squeeze()  # Remover dimensão extra, se houver

In [ ]:
# Métricas de avaliação
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

test_mse = mean_squared_error(y_train, y_pred)
test_mae = mean_absolute_error(y_train, y_pred)
test_r2 = r2_score(y_train, y_pred)

print(f'Test Mean Squared Error (MSE): {test_mse}')
print(f'Test Mean Absolute Error (MAE): {test_mae}')
print(f'Test R^2 Score: {test_r2}')

In [ ]:
# Métricas de avaliação
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

test_mse = mean_squared_error(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print(f'Test Mean Squared Error (MSE): {test_mse}')
print(f'Test Mean Absolute Error (MAE): {test_mae}')
print(f'Test R^2 Score: {test_r2}')